## HTML Template

### 1. Frontend

<!DOCTYPE HTML>
<html>
Temperature[degC]: 24.40<br />Humidity[%]: 64.00<br />heater_onOff[0:on,1:off]: 0<br />T_set: 30.00<br />is_sitting[0:true,1:false]: 1<br />T00_is_chair: 23.80<br />T01_is_chair: 23.90<br />T02_is_chair: 23.80<br />PID Output: 100.00<br /></html>


### 2. Source code

Jede Datenzeile wird durch `<br />` getrennt

```
<!DOCTYPE HTML>
<html>
Temperature[degC]: 24.40<br />Humidity[%]: 64.00<br />heater_onOff[0:on,1:off]: 0<br />T_set: 30.00<br />is_sitting[0:true,1:false]: 1<br />T00_is_chair: 23.80<br />T01_is_chair: 23.90<br />T02_is_chair: 23.80<br />PID Output: 100.00<br /></html>
```

### 3. Beispielcode

Hier werden die Daten in einer Liste gespeichert, alternativ können sie auch in dict gespeichert werden.

In [1]:
# -*- coding: utf-8 -*-
# =================================
# Python 3
# Jupyter Notebook 6.1.4
# Author: hqrrr
# Date of last update: Jan. 2021
# =================================

import urllib  # get data from an URL

# settings of urllib
link = "http://192.168.0.99/"


# reset all lists
valueRead_list = []
valueRead_list_1 = []

# open link
f = urllib.request.urlopen(link)


# read data and processing
valueRead = f.read()  # type: bytes

valueRead_str = str(valueRead.decode("utf-8"))  # type bytes -> string
print("valueRead_str: " + valueRead_str)

valueRead_str = valueRead_str.strip('<!DOCTYPE HTML>/html').strip().strip('<br />/html').strip()  # delete useless characters
print("valueRead_str: " + valueRead_str)

valueRead_list = valueRead_str.split("<br />")  # split into a list according to <br />
print("valueRead_list: ")
print(valueRead_list)

valueRead_list_1 = [i.split(": ") for i in valueRead_list] # split values and labels in a new list
print("valueRead_list_1: ")
print(valueRead_list_1)

valueRead_str: <!DOCTYPE HTML>
<html>
Temperature[degC]: 24.50<br />Humidity[%]: 64.00<br />heater_onOff[0:on,1:off]: 0<br />T_set: 30.00<br />is_sitting[0:true,1:false]: 1<br />T00_is_chair: 23.70<br />T01_is_chair: 23.90<br />T02_is_chair: 23.80<br />PID Output: 100.00<br /></html>

valueRead_str: Temperature[degC]: 24.50<br />Humidity[%]: 64.00<br />heater_onOff[0:on,1:off]: 0<br />T_set: 30.00<br />is_sitting[0:true,1:false]: 1<br />T00_is_chair: 23.70<br />T01_is_chair: 23.90<br />T02_is_chair: 23.80<br />PID Output: 100.00
valueRead_list: 
['Temperature[degC]: 24.50', 'Humidity[%]: 64.00', 'heater_onOff[0:on,1:off]: 0', 'T_set: 30.00', 'is_sitting[0:true,1:false]: 1', 'T00_is_chair: 23.70', 'T01_is_chair: 23.90', 'T02_is_chair: 23.80', 'PID Output: 100.00']
valueRead_list_1: 
[['Temperature[degC]', '24.50'], ['Humidity[%]', '64.00'], ['heater_onOff[0:on,1:off]', '0'], ['T_set', '30.00'], ['is_sitting[0:true,1:false]', '1'], ['T00_is_chair', '23.70'], ['T01_is_chair', '23.90'], ['

### 4. Control logic (Arduino)

if 

heater_onOff = 0 (controlled by user on Arduino_Desk)

and 

T00 & T01 & T02 (readed from Arduino_chair) < T_set (controlled by user on Arduino_Desk)

and

is_sitting = 0 (readed from Arduino_chair)

->

Heizmatte anschalten

->

PID Output ist der Power Output von PI Regler (zwischen 0 - 100, 0:min., 1:max.)

->

Der Output-Wert kontrolliert die Zeit, wie lange die Heizmatte an oder ausgeschaltet ist. Periode: 5s

#### Arduino Code: (do not change & run in jupyter notebook)

In [ ]:
void PID_control() {
  // PID controller, read Input from sensor(received from Master), compute, write Output to actuator
  Input = T_room;
  Setpoint = T_set;
  heaterPID.Compute();
  // Set Power (Output) within 0 and 100 %
  Output = constrain(Output, 0, 100);
  //analogWrite(3, Output);
  
  // Output for serial plotter
  Serial.print("PID Input: ");
  Serial.print(Input);
  Serial.print("   ");
  Serial.print("PID Output: ");       
  Serial.print(Output);
  Serial.print("   ");
  Serial.print("PID Setpoint: ");
  Serial.println(Setpoint);  

  // Prinzip PI Regler ->
  // - On        ______         _______           ________
  // - Off _____|      |_______|       |_________|        |______
  //            /------/               /---------/
  //         heatingOn_time          heatingOff_time
  // z.B. PID Output = 0 -> heatingOn_time = 0 und heatingOff_time = 5000 (Periode: 5 sec.)
  //      PID Output = 100 -> heatingOn_time = 5000 und heatingOff_time = 0.
  
  heatingOn_time = 5000 * Output / 100;
  heatingOff_time = 5000 - heatingOn_time;

  if (ds00_temperature < T_set && ds01_temperature < T_set && ds02_temperature < T_set && is_sitting == 0 && heater_onOff == 0){
    
    analogWrite(mosfetPin, mosfetOn); // Heizmatte anschalten
    delay(heatingOn_time); // die Zeit ist abhängig von PID Output
    analogWrite(mosfetPin, mosfetOff);
    delay(heatingOff_time);
  }
  else {
    analogWrite(mosfetPin, mosfetOff);
    delay(5000);
  }
}